In [106]:
import pandas as pd
import pyodbc
class ReadSQL():

    def __init__(self, DB, strQuery):
        self.DB = DB
        self.strQuery = strQuery
    def run(self):
        connection = pyodbc.connect(
        'DRIVER={ODBC Driver 18 for SQL Server};'
        'SERVER=TOM;'
        'DATABASE=' + self.DB + ';'
        'UID=TOM;'
        'PWD=Silversea;'
        'TrustServerCertificate=yes;'   
        )
        df = pd.read_sql_query(self.strQuery, connection)
        return df
class PivotbyList():
    def __init__(self, df, li, index, columns):
        self.df = df
        self.li = li
        self.index = index
        self.columns = columns
    def run(self):
        filtered_df = self.df[self.df[self.columns].isin(self.li)]
        pivot_df = pd.pivot_table(
            filtered_df,
            index= self.index,
            columns=self.columns,
            values=[col for col in self.df.columns if col not in [self.index, self.columns]],
            aggfunc='first' 
        )

        pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([[col for col in self.df.columns if col not in [self.index, self.columns]], self.li]), fill_value=None)

        pivot_df.columns = [f"{addr}_{col}" for col, addr in pivot_df.columns]

        pivot_df.reset_index(inplace=True)
        return pivot_df

In [107]:
li_CRS = ['Tax in charge','Audit in charge','Manager','Primary Partner','Bill Manager','Fund Manager','Tax Manager','Audit Manager','Audit Staff','Fund in charge','Associate']
li_Address = ['Home 2','Billing/AR','Business','Home','Other','Business 2']
li_Phone = ['Assistant','Home Fax','Home','Mobile','HOME 3','Business','Business Fax','Other Fax','Business 3','Mobile 2','Home 2','Spouse Mobile','Spouse Business','Spouse Home','Pager','Spouse Fax','Other','Business 2']
li_Email = ['Clients Portal Admin','Home','Business','Home 2','Spouse Business','Spouse Home','Other','Billing/AR Issues','Business 2']
li_choose_cols = ['ClientIdSubId','LastInv', 'ClientGroupName', 'BillingGroupName', 'No. of Note', 'ClientNote', 'No. of ClientContact', 'ClientContact']

for i in li_CRS:
    li_choose_cols.append(i + '_StaffName')
for i in li_Phone:
    li_choose_cols.append(i + '_PhoneNumber')
for i in li_Email:
    li_choose_cols.append(i + '_EmailAddress')
for j in li_Address:

    for i in ['AddressLine1','AddressLine2','AddressLine3','CityName','PostalCode','CountryCode','AptOrSuiteNumber','StateProvinceCode']:
        li_choose_cols.append(j + '_' + i)

In [108]:
Spiegel_Client = "SELECT C.ClientIdent,ClientIdSubId,ISNULL(I.lastinv, 'No Inv') LastInv, ClientGroupName, BillingGroupName FROM CLIENT C LEFT JOIN ( 	SELECT ClientIdent, ClientGroupName FROM CLIENTGROUP CG 	WHERE FinancialReportingGroupFlag = 'T' ) CGF ON CGF.ClientIdent = C.ClientIdent LEFT JOIN ( 	SELECT ClientIdent,BillingGroupName FROM CLIENTPRACTICE 	where BillingGroupName <> '' ) CBG ON CBG.ClientIdent = C.ClientIdent LEFT JOIN ( 	select ClientIdent, FORMAT(MAX(InvoiceDateTime),'MM/dd/yyyy') lastinv from INVOICE 	WHERE StdWIPAmount + AdjustmentAmount + ProgressBilledAmount - ProgressApplyAmount + TaxAmount + ProgressTaxAmount - TaxAppAmount - ProgressTaxAppAmount <> 0 	AND StatusCode = 1 	GROUP BY ClientIdent ) I ON I.ClientIdent = C.ClientIdent"
ACS_Client = "SELECT C.ClientIdent,ClientIdSubId,ISNULL(I.lastinv, 'No Inv') LastInv, ClientGroupName, BillingGroupName FROM CLIENT C LEFT JOIN ( 	SELECT ClientIdent, ClientGroupName FROM CLIENTGROUP CG 	WHERE FinancialReportingGroupFlag = 'T' ) CGF ON CGF.ClientIdent = C.ClientIdent LEFT JOIN ( 	SELECT ClientIdent,BillingGroupName FROM CLIENTPRACTICE 	where BillingGroupName <> '' ) CBG ON CBG.ClientIdent = C.ClientIdent LEFT JOIN ( 	select ClientIdent, FORMAT(MAX(InvoiceDateTime),'MM/dd/yyyy') lastinv from INVOICE 	WHERE StdWIPAmount + AdjustmentAmount + ProgressBilledAmount - ProgressApplyAmount + TaxAmount + ProgressTaxAmount - TaxAppAmount - ProgressTaxAppAmount <> 0 	AND StatusCode = 1 	GROUP BY ClientIdent ) I ON I.ClientIdent = C.ClientIdent where C.ClientOfficeName = 'Spiegel Office'"
Spiegel_Address = "SELECT ReferenceIdent ClientIdent, FirmAddressTypeLabelName, AddressLine1, AddressLine2, AddressLine3, CityName, PostalCode, CountryCode, AptOrSuiteNumber, StateProvinceCode FROM CLIENTADDRESS where AddressIdentCategoryType = 'Client' and  ReferenceIdentType = 'ClientIntID'"
ACS_Address = "SELECT ReferenceIdent as ClientIdent, FirmAddressTypeLabelName, AddressLine1, AddressLine2, AddressLine3, CityName, PostalCode, CountryCode, AptOrSuiteNumber, StateProvinceCode FROM CLIENTADDRESS where AddressIdentCategoryType = 'Client' and ReferenceIdentType = 'ClientIntID'"
Spiegel_CRS = "SELECT ClientIdent, FirmClientStaffAssignmentName, StaffName FROM CLIENTCRS CRS LEFT JOIN STAFF S ON S.StaffIdent = CRS.StaffIdent"
ACS_CRS = "SELECT ClientIdent, FirmClientStaffAssignmentName, StaffName FROM CLIENTCRS CRS LEFT JOIN STAFF S ON S.StaffIdent = CRS.StaffIdent"
Spiegel_ClientNote = "SELECT ClientIdent, COUNT(*) [No. of Note] , (SELECT STUFF((SELECT DISTINCT '|' + IIF(ClientNoteSubject='',ClientNoteText, ClientNoteSubject + ' - ' + ClientNoteText) FROM CLIENTNOTE  CN WHERE DeleteFlag = 'F' AND CN.ClientIdent = CLIENTNOTE.ClientIdent FOR XML PATH('')),1,1,''))  ClientNote FROM CLIENTNOTE GROUP BY ClientIdent"
ACS_ClientNote = "SELECT ClientIdent, COUNT(*) [No. of Note] , (SELECT STUFF((SELECT DISTINCT '|' + ClientNoteText FROM CLIENTNOTE  CN WHERE DeleteFlag = 'F' AND CN.ClientIdent = CLIENTNOTE.ClientIdent FOR XML PATH('')),1,1,''))  ClientNote FROM CLIENTNOTE GROUP BY ClientIdent"
Spiegel_ClientContact = "SELECT * FROM SS_Contact"
ACS_ClientContact = "SELECT * FROM SS_Contact"
Spiegel_ClientPhone = "SELECT ReferenceIdent as ClientIdent, FirmPhoneTypeLabelName, IIF(PhoneNumber='',null,PhoneNumber) PhoneNumber FROM CLIENTPHONE where PhoneCategoryType = 'Client' and ReferenceIdentType = 'ClientIntID' and DeleteFlag = 'F'"
ACS_ClientPhone = "SELECT ReferenceIdent as ClientIdent, FirmPhoneTypeLabelName, IIF(PhoneNumber='',null,PhoneNumber) PhoneNumber FROM CLIENTPHONE where PhoneCategoryType = 'Client' and ReferenceIdentType = 'ClientIntID' and DeleteFlag = 'F'"
Spiegel_ClientEmail = "SELECT ReferenceIdent as ClientIdent, FirmEmailTypeName, IIF(EmailAddress='',null,EmailAddress) EmailAddress FROM CLIENTEMAIL where ReferenceIdentType = 'ClientIntId' and EmailIdentCategoryType = 'Client' and DeleteFlag = 'F'"
ACS_ClientEmail = "SELECT ReferenceIdent as ClientIdent, FirmEmailTypeName, IIF(EmailAddress='',null,EmailAddress) EmailAddress FROM CLIENTEMAIL where ReferenceIdentType = 'ClientIntId' and EmailIdentCategoryType = 'Client' and DeleteFlag = 'F'"

In [109]:
df_Spiegel_Client = ReadSQL('CCH_Spiegel_11.09.24', Spiegel_Client).run()
df_Spiegel_Address = ReadSQL('CCH_Spiegel_11.09.24', Spiegel_Address).run()
df_Spiegel_Address = PivotbyList(df_Spiegel_Address, li_Address, 'ClientIdent', 'FirmAddressTypeLabelName').run()
df_Spiegel_CRS = ReadSQL('CCH_Spiegel_11.09.24', Spiegel_CRS).run()
df_Spiegel_CRS = PivotbyList(df_Spiegel_CRS, li_CRS, 'ClientIdent', 'FirmClientStaffAssignmentName').run()
df_Spiegel_ClientNote = ReadSQL('CCH_Spiegel_11.09.24', Spiegel_ClientNote).run()
df_Spiegel_ClientContact = ReadSQL('CCH_Spiegel_11.09.24', Spiegel_ClientContact).run()
df_Spiegel_ClientPhone = ReadSQL('CCH_Spiegel_11.09.24', Spiegel_ClientPhone).run()
df_Spiegel_ClientPhone = PivotbyList(df_Spiegel_ClientPhone, li_Phone, 'ClientIdent', 'FirmPhoneTypeLabelName').run()
df_Spiegel_ClientEmail = ReadSQL('CCH_Spiegel_11.09.24', Spiegel_ClientEmail).run()
df_Spiegel_ClientEmail = PivotbyList(df_Spiegel_ClientEmail, li_Email, 'ClientIdent', 'FirmEmailTypeName').run()
df_Spiegel_List = [df_Spiegel_ClientNote,df_Spiegel_ClientContact,df_Spiegel_CRS, df_Spiegel_ClientPhone, df_Spiegel_ClientEmail, df_Spiegel_Address]
for df_to_join in df_Spiegel_List:
    df_Spiegel_Client = pd.merge(df_Spiegel_Client, df_to_join, on='ClientIdent', how='left')
df_Spiegel_Client = df_Spiegel_Client[li_choose_cols]
df_Spiegel_Client = df_Spiegel_Client.fillna('')
df_Spiegel_Client['No. of Note'] = df_Spiegel_Client['No. of Note'].replace('','0')
df_Spiegel_Client['No. of ClientContact'] = df_Spiegel_Client['No. of ClientContact'].replace('','0')
for i in df_Spiegel_Client.columns:
    if i in ['No. of ClientContact','No. of Note']:
        df_Spiegel_Client[i] = df_Spiegel_Client[i].astype(int)
    else:
        df_Spiegel_Client[i] = df_Spiegel_Client[i].astype('string')
li_cols_Spiegel = []
for i in li_choose_cols:
    li_cols_Spiegel.append('Spiegel_' + i)
df_Spiegel_Client = df_Spiegel_Client.rename(columns=dict(zip(li_choose_cols, li_cols_Spiegel)))
df_Spiegel_Client.to_csv('test.csv',index=False)



C:\Users\tomda\AppData\Local\Temp\ipykernel_24000\335444848.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(self.strQuery, connection)
C:\Users\tomda\AppData\Local\Temp\ipykernel_24000\335444848.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(self.strQuery, connection)
C:\Users\tomda\AppData\Local\Temp\ipykernel_24000\335444848.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(self.strQuery, connection)
C:\Users\tomda\AppData\Local\Temp\ipyker

In [110]:
df_ACS_Client = ReadSQL('CCH_ACS_11.09.24', ACS_Client).run()
df_ACS_Address = ReadSQL('CCH_ACS_11.09.24', ACS_Address).run()
df_ACS_Address = PivotbyList(df_ACS_Address, li_Address, 'ClientIdent', 'FirmAddressTypeLabelName').run()
df_ACS_CRS = ReadSQL('CCH_ACS_11.09.24', ACS_CRS).run()
df_ACS_CRS = PivotbyList(df_ACS_CRS, li_CRS, 'ClientIdent', 'FirmClientStaffAssignmentName').run()
df_ACS_ClientNote = ReadSQL('CCH_ACS_11.09.24', ACS_ClientNote).run()
df_ACS_ClientContact = ReadSQL('CCH_ACS_11.09.24', ACS_ClientContact).run()
df_ACS_ClientPhone = ReadSQL('CCH_ACS_11.09.24', ACS_ClientPhone).run()
df_ACS_ClientPhone = PivotbyList(df_ACS_ClientPhone, li_Phone, 'ClientIdent', 'FirmPhoneTypeLabelName').run()
df_ACS_ClientEmail = ReadSQL('CCH_ACS_11.09.24', ACS_ClientEmail).run()
df_ACS_ClientEmail = PivotbyList(df_ACS_ClientEmail, li_Email, 'ClientIdent', 'FirmEmailTypeName').run()
df_ACS_List = [df_ACS_ClientNote,df_ACS_ClientContact,df_ACS_CRS, df_ACS_ClientPhone, df_ACS_ClientEmail, df_ACS_Address]
for df_to_join in df_ACS_List:
    df_ACS_Client = pd.merge(df_ACS_Client, df_to_join, on='ClientIdent', how='left')
df_ACS_Client = df_ACS_Client[li_choose_cols]
df_ACS_Client = df_ACS_Client.fillna('')
df_ACS_Client['No. of Note'] = df_ACS_Client['No. of Note'].replace('','0')
df_ACS_Client['No. of ClientContact'] = df_ACS_Client['No. of ClientContact'].replace('','0')
for i in df_ACS_Client.columns:
    if i in ['No. of ClientContact','No. of Note']:
        df_ACS_Client[i] = df_ACS_Client[i].astype(int)
    else:
        df_ACS_Client[i] = df_ACS_Client[i].astype('string')
li_cols_ACS = []
for i in li_choose_cols:
    li_cols_ACS.append('ACS_' + i)
df_ACS_Client = df_ACS_Client.rename(columns=dict(zip(li_choose_cols, li_cols_ACS)))

C:\Users\tomda\AppData\Local\Temp\ipykernel_24000\335444848.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(self.strQuery, connection)
C:\Users\tomda\AppData\Local\Temp\ipykernel_24000\335444848.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(self.strQuery, connection)
C:\Users\tomda\AppData\Local\Temp\ipykernel_24000\335444848.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(self.strQuery, connection)
C:\Users\tomda\AppData\Local\Temp\ipyker

In [111]:
df_merged = pd.merge(df_Spiegel_Client, df_ACS_Client, left_on='Spiegel_ClientIdSubId', right_on='ACS_ClientIdSubId', how='outer')
li_cols_merged = []
for i in li_choose_cols:
    li_cols_merged.append('ACS_' + i)
    li_cols_merged.append('Spiegel_' + i)
df_merged = df_merged[li_cols_merged]
for i in li_cols_merged:
    if 'No. of Note' in i or 'No. of ClientContact' in i:
        df_merged[i] = df_merged[i].fillna('0')
        df_merged[i] = df_merged[i].astype(int)
    else:
        df_merged[i] = df_merged[i].fillna('')
        df_merged[i] = df_merged[i].astype('string')
df_merged['Status'] = 'Match'
df_merged['Unmatch Fields'] = ''
for i in range(df_merged.shape[0]):
    strStatus = ''
    if df_merged.at[i,'Spiegel_ClientIdSubId'] == '' and df_merged.at[i,'ACS_ClientIdSubId'] != '':
        df_merged.at[i,'Status'] = 'Not found in Spiegel'
    elif df_merged.at[i,'Spiegel_ClientIdSubId'] != '' and df_merged.at[i,'ACS_ClientIdSubId'] == '':
        df_merged.at[i,'Status'] = 'Not found in ACS'
    elif df_merged.at[i,'Spiegel_LastInv'] == 'No Inv' and df_merged.at[i,'ACS_LastInv'] != 'No Inv' and df_merged.at[i,'Spiegel_ClientIdSubId'] != '' and df_merged.at[i,'ACS_ClientIdSubId'] != '':
        df_merged.at[i,'Status'] = 'No Inv in Spiegel'
    elif df_merged.at[i,'ACS_LastInv'] == 'No Inv' and df_merged.at[i,'Spiegel_LastInv'] != 'No Inv' and df_merged.at[i,'Spiegel_ClientIdSubId'] != '' and df_merged.at[i,'ACS_ClientIdSubId'] != '':
        df_merged.at[i,'Status'] = 'No Inv in ACS'
    elif df_merged.at[i,'ACS_LastInv'] == 'No Inv' and df_merged.at[i,'Spiegel_LastInv'] == 'No Inv' and df_merged.at[i,'Spiegel_ClientIdSubId'] != '' and df_merged.at[i,'ACS_ClientIdSubId'] != '':
        df_merged.at[i,'Status'] = 'No Inv in ACS/Spiegel'
    else:
        df_merged.at[i,'Status'] = 'See "Unmatch Fields" column'
for i in range(df_merged.shape[0]):
    strStatus = ''
    if df_merged.at[i,'Spiegel_ClientIdSubId'] == df_merged.at[i,'ACS_ClientIdSubId']:
        for j in li_choose_cols:
            if df_merged.at[i,'ACS_' + j] != df_merged.at[i,'Spiegel_' + j] and j != 'LastInv':
                strStatus = strStatus + ',' + j
        if strStatus != '':
            df_merged.at[i,'Unmatch Fields'] = strStatus        
df_merged = df_merged[['Status', 'Unmatch Fields'] + [col for col in df_merged.columns if col not in ['Status', 'Unmatch Fields']]]
# Refixed Status Column
for i in range(df_merged.shape[0]):
    if df_merged.at[i,'Unmatch Fields'] == '' and 'No Inv' in df_merged.at[i,'Status']:
        df_merged.at[i,'Status'] = 'Match but ' + df_merged.at[i,'Status']
    if df_merged.at[i,'Unmatch Fields'] == '' and df_merged.at[i,'Status'] == 'See "Unmatch Fields" column':
        df_merged.at[i,'Status'] = 'Match'
df_merged.to_csv('test.csv',index=False)